This will  help you test API

 uvicorn api.main:app --reload

http://127.0.0.1:8000/docs#/default/predict_predict_post

In [1]:
import numpy as np
import joblib
import json
import os
import random

# ----------------------------------------------------
# SET RANDOMNESS (comment seed for true randomness)
# ----------------------------------------------------
random.seed()        # different each run
# random.seed(42)    # uncomment for reproducible runs

# ----------------------------------------------------
# RESOLVE PROJECT ROOT (NOTEBOOK-SAFE)
# ----------------------------------------------------
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
MODEL_DIR = os.path.join(PROJECT_ROOT, "models")

SCALER_PATH = os.path.join(MODEL_DIR, "scaler.pkl")
AUTOENCODER_PATH = os.path.join(MODEL_DIR, "autoencoder.pkl")
X_VAL_PATH = os.path.join(MODEL_DIR, "X_val_scaled.npy")

# ----------------------------------------------------
# LOAD ARTIFACTS
# ----------------------------------------------------
scaler = joblib.load(SCALER_PATH)
autoencoder = joblib.load(AUTOENCODER_PATH)
X_val = np.load(X_VAL_PATH)

feature_names = list(scaler.feature_names_in_)

# ----------------------------------------------------
# COMPUTE RECONSTRUCTION ERRORS
# ----------------------------------------------------
X_val_pred = autoencoder.predict(X_val)
reconstruction_errors = ((X_val - X_val_pred) ** 2).mean(axis=1)

THRESHOLD = np.percentile(reconstruction_errors, 99.3)

# ----------------------------------------------------
# HELPER → API INPUT
# ----------------------------------------------------
def to_api_input(idx):
    x_scaled = X_val[idx].reshape(1, -1)
    x_raw = scaler.inverse_transform(x_scaled)[0]
    return {
        "features": {
            fname: float(value)
            for fname, value in zip(feature_names, x_raw)
        }
    }

# ----------------------------------------------------
# RANDOMLY SELECT LOW / MEDIUM / HIGH
# ----------------------------------------------------
low_candidates = np.where(reconstruction_errors < THRESHOLD * 0.4)[0]
medium_candidates = np.where(
    (reconstruction_errors >= THRESHOLD * 0.8) &
    (reconstruction_errors < THRESHOLD)
)[0]
high_candidates = np.where(reconstruction_errors > THRESHOLD * 2.0)[0]

low_idx = random.choice(low_candidates)
medium_idx = random.choice(medium_candidates)
high_idx = random.choice(high_candidates)

# ----------------------------------------------------
# PRINT API-READY JSON
# ----------------------------------------------------
print("\n================ LOW RISK =================")
print(json.dumps(to_api_input(low_idx), indent=2))

print("\n=============== MEDIUM RISK ===============")
print(json.dumps(to_api_input(medium_idx), indent=2))

print("\n================ HIGH RISK ================")
print(json.dumps(to_api_input(high_idx), indent=2))



================ LOW RISK =================
{
  "features": {
    "Time": 42851.0,
    "Amount": 28.0,
    "V1": 1.34207150900445,
    "V2": -0.397466527418524,
    "V3": 0.192588345066391,
    "V4": -0.727975464075449,
    "V5": -0.843419372670587,
    "V6": -1.09530355528723,
    "V7": -0.233875067921705,
    "V8": -0.19045264824520203,
    "V9": -1.3227255622997498,
    "V10": 0.79995812345115,
    "V11": 1.70972641142748,
    "V12": 0.15070833874401,
    "V13": -0.177630059222702,
    "V14": 0.31171358940801,
    "V15": -0.180463390325282,
    "V16": 1.08070122982848,
    "V17": 0.199594286363536,
    "V18": -1.4641269454704,
    "V19": 1.00550270187644,
    "V20": 0.089419290536828,
    "V21": -0.131548870629957,
    "V22": -0.600058851801328,
    "V23": 0.130490356464332,
    "V24": 0.512676184225274,
    "V25": 0.278249052548441,
    "V26": -0.561279842650082,
    "V27": -0.0240052928209816,
    "V28": 0.0086174070706427,
    "home_lat": 52.20685589450593,
    "home_lon": 11.28